# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119678e+02     1.555188e+00
 * time: 0.12992501258850098
     1     1.036344e+01     8.794576e-01
 * time: 0.9579598903656006
     2    -1.157328e+01     9.910745e-01
 * time: 1.1047539710998535
     3    -3.385133e+01     7.375764e-01
 * time: 1.252047061920166
     4    -4.713544e+01     5.760360e-01
 * time: 1.3885259628295898
     5    -5.674477e+01     2.134433e-01
 * time: 1.5302529335021973
     6    -5.962189e+01     1.784529e-01
 * time: 1.6428070068359375
     7    -6.082755e+01     8.426401e-02
 * time: 1.7565970420837402
     8    -6.134592e+01     4.061864e-02
 * time: 1.8630380630493164
     9    -6.164547e+01     3.911930e-02
 * time: 1.9758670330047607
    10    -6.183660e+01     2.930725e-02
 * time: 2.0832479000091553
    11    -6.197774e+01     2.502106e-02
 * time: 2.196089029312134
    12    -6.205703e+01     2.607163e-02
 * time: 2.309519052505493
    13    -6.211815e+01     1.726641e-02
 * time: 2.414381027221

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671070
    AtomicLocal         -18.8557698
    AtomicNonlocal      14.8522659
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485379 
    Xc                  -19.3336823

    total               -62.261666458719
